# Demo for Starboard with pyodide

## Import `pyqe` library

This is the [pyodide](https://pyodide.org/en/stable/) version of the `pyqe` library. This library uses the `await` syntax in places where API calls (via request library) are made.

In [ ]:
from pyqe import *

## Define authenticated query with name

In [ ]:
query = Query('Number_of_female_patients_born_in_1930s_not_from_California_or_Florida')

## Select study and config

In [ ]:
await query.set_study('eb702c85-ffbd-4507-8061-4b0f458a9b11')

## Define Patient Filter

In [ ]:
female_patient = Person.Patient()
female_constraint = Constraint().add(Expression(ComparisonOperator.EQUAL, 'Female'))
female_patient.add_gender([female_constraint])

not_ca_state_constraint = Constraint().add(Expression(ComparisonOperator.NOT_EQUAL, 'CA'))
not_fl_state_constraint = Constraint().add(Expression(ComparisonOperator.NOT_EQUAL, 'FL'))
female_patient.add_state([not_ca_state_constraint, not_fl_state_constraint])

year_of_birth_constraint = Constraint()
year_of_birth_constraint.add(Expression(ComparisonOperator.MORE_THAN_EQUAL, '1930'))
year_of_birth_constraint.add(Expression(ComparisonOperator.LESS_THAN_EQUAL, '1939'))
female_patient.add_yearofbirth([year_of_birth_constraint])

query.add_filters([female_patient])
request = query.get_patient_count_filter()

## Get patient count with request

In [ ]:
patient_count = await Result().get_patient_count(request)
print(f'\nTotal female patients born in 1930s not from California or Florida: {patient_count}')

## Add condition occurrence for low back pain or type 2 diabetes, generate request & get patient count

In [ ]:
condition = Interactions.ConditionOccurrence('')
low_back_pain_constraint = Constraint().add(Expression(ComparisonOperator.EQUAL, 'Low back pain'))
type_2_diabetes_constraint = Constraint().add(Expression(ComparisonOperator.EQUAL, 'Type 2 diabetes mellitus'))
condition.add_condition_name([low_back_pain_constraint, type_2_diabetes_constraint])
query.add_filters([condition])

request = query.get_patient_count_filter()
patient_count = await Result().get_patient_count(request)
print(f'\nTotal female patients born in 1930s not from California or Florida with Low back pain or Type 2 diabetes mellitus: {patient_count}')

## Create and define another query

In [ ]:
concept_query = Query('Number_of_male_patients_with_outpatient_visits')

male_patient = Person.Patient()
male_patient.add_concept_set(ConceptSet('Male', Domain.GENDER, ['M']))

outpatient_visit = Interactions.Visit('Outpatient Visit')
outpatient_visit.add_concept_set(ConceptSet('Outpatient Visit', Domain.VISIT, ['OP']))

## Generate request & get patient count

In [ ]:
male_outpatient_visit_group = CriteriaGroup(MatchCriteria.ALL,
                                            [male_patient, outpatient_visit])
concept_query.add_criteria_group(male_outpatient_visit_group)
concept_request = concept_query.get_patient_count_filter()

concept_result = Result()
concept_patient_count = await concept_result.get_patient_count(concept_request)
print(f'\nTotal male patient with outpatient visits: {concept_patient_count}')

## Create a phenotype example (sleep apnea)

In [ ]:
conceptQuery = ConceptQuery()

# sleep_apnea_dx_snomed_concept_codes = ['73430006', '41975002', '79280005', '78275009']
sleep_apnea_dx_concept_ids = ['313459', '434172', '439150', '442588']

# sleep_apnea_device_hcpcs_concept_codes = ['A7034', 'A9270', 'E0470', 'E0471', 'E0601']
sleep_apnea_device_concept_ids = ['2615264', '2615309', '2616639', '2616640', '2616666']

# sleep_apnea_snomed_procedure_concept_code = '60554003'
sleep_apnea_snomed_procedure_concept_id = ['4189015']

## Get patient count with request

In [ ]:
phenotype_query = Query('Total_Patients_With_Sleep_Apnea')

sleep_apnea_dx = ConceptSet('Sleep Apnea Dx', Domain.CONDITION, sleep_apnea_dx_concept_ids)
sleep_apnea_dx_condition = Interactions.ConditionOccurrence(sleep_apnea_dx.name)
sleep_apnea_dx_condition.add_concept_set(sleep_apnea_dx)

sleep_apnea_device = ConceptSet('Sleep Apnea Device', Domain.DEVICE, sleep_apnea_device_concept_ids)
sleep_apnea_device_exposure = Interactions.DeviceExposure(sleep_apnea_device.name)
sleep_apnea_device_exposure.add_concept_set(sleep_apnea_device)


phenotype_query.add_criteria_group(CriteriaGroup(MatchCriteria.ALL, [sleep_apnea_dx_condition, sleep_apnea_device_exposure]))
phenotype_request = phenotype_query.get_patient_count_filter()

phenotype_patient_count = await Result().get_patient_count(phenotype_request)
print(
    f'\nTotal patients with Sleep Apnea condition and device exposure: {phenotype_patient_count}')